In [ ]:
from unet import Unet
from n2v import N2VConfig, N2VDataGenerator, DataGenerator
from os.path import join
import numpy as np

Traditional training from noisy to clean

In [ ]:
file_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-lowsnr.tif'
gt_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-highsnr.tif'
config = N2VConfig(file_paths=[file_path], ground_truth_paths=[gt_path])
unet = Unet(config, n_depth=2)

In [ ]:
traditional_data_generator=DataGenerator(file_path,gt_path,batch_size=1,validation_split=0.1)
history=unet.train(traditional_data_generator, epochs=10)

In [ ]:
n2v_data_generator=N2VDataGenerator(config)
result=unet.evaluate(n2v_data_generator, divide=1, save_dir="restored_traditional")
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print('duration:',result['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

N2V training on one single image

In [ ]:
# file_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-20x-noise1-lowsnr.tif'
# gt_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-20x-noise1-highsnr.tif'
# with Image.open(file_path) as img,Image.open(gt_path) as gt:
#     img.save("example.tif")
#     gt.save("example_gt.tif")

In [ ]:
config = N2VConfig(file_paths=["example.tif"],
                   ground_truth_paths=["example_gt.tif"],
                   patch_shape=(64, 64),
                   patches_per_batch=128,
                   data_augmentation=True,
                   epochs=100)
unet = Unet(config, n_depth=2)


In [ ]:
n2v_data_generator = N2VDataGenerator(config)
history=unet.train(n2v_data_generator)

In [ ]:
result=unet.evaluate(n2v_data_generator, divide=1, save_dir="restored_0")
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print(result['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

In [ ]:


dataset_dir = r'C:\Users\bioimaging1\Desktop\Deep learning image\actin'
noisy_inputs = ["actin-20x-noise1-lowsnr.tif",
            # "actin-confocal-lowsnr.tif",
            # "actin-60x-noise1-lowsnr.tif",
            # "membrane-lowsnr.tif",
            # "mito-20x-noise1-lowsnr.tif",
            # "mito-60x-noise1-lowsnr.tif",
            # "mito-confocal-lowsnr.tif",
            # "nucleus-lowsnr.tif",
            # "mito-60x-noise2-lowsnr.tif"
            ]
ground_truth = ["actin-20x-noise1-highsnr.tif",
            # "actin-confocal-highsnr.tif",
            # "actin-60x-noise1-highsnr.tif",
            # "membrane-highsnr.tif",
            # "mito-20x-noise1-highsnr.tif",
            # "mito-60x-noise1-highsnr.tif",
            # "mito-confocal-highsnr.tif",
            # "nucleus-highsnr.tif",
            # "mito-60x-noise2-highsnr.tif"
            ] 
file_paths = [join(dataset_dir, dataset) for dataset in noisy_inputs]
ground_truth_paths = [join(dataset_dir, dataset) for dataset in ground_truth]
config = N2VConfig(file_paths, ground_truth_paths,
                   patch_shape=(128, 128),
                   validation_split=0.1,
                   patches_per_batch=32,
                   data_augmentation=True,
                   epochs=10)

n2v_data_generator = N2VDataGenerator(config)
unet = Unet(config, model_summary=False)

In [ ]:
config = N2VConfig(file_paths, ground_truth_paths,
                   patch_shape=(128, 128),
                   validation_split=0.1,
                   patches_per_batch=32,
                   data_augmentation=True,
                   epochs=20)

n2v_data_generator = N2VDataGenerator(config)
history = unet.train(n2v_data_generator, early_stopping_patience=5, reduce_lr_patience=2                                                                                )
# 可以在Tensorboard查看训练损失和验证损失的变化情况
# cmd: tensorboard --logdir tensorboard 

In [ ]:
# unet.model.load_weights("ckpt/best")
result=unet.evaluate(n2v_data_generator, divide=8, save_dir="restored_images_test1")


In [ ]:
unet.model.load_weights("ckpt/best")
result=unet.evaluate(n2v_data_generator, divide=1, save_dir=None)
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print(result['duration'])
print('ssim:',ssim.mean(),'\tpsnr:', psnr.mean())
print('old_ssim:',old_ssim.mean(),'\told_psnr:', old_psnr.mean())

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
import matplotlib.pyplot as plt
epochs=range(len(loss))
plt.plot(loss,label='loss')
plt.plot(val_loss,label='val_loss')
plt.xlabel("epoch")
plt.legend()
